In [2]:
import pandas as pd
import numpy as np

In [3]:
datab = pd.read_csv('Mouvements_wiki_cleaned.csv')

In [4]:
#je recopie les fonctions de prédiction que je modifie quelque peu (rajout option 2) 

Je recharge et modifie les fonctions utiles à la prédiction.

In [5]:
#juste pour ressortir toutes les prédictions "ex aequo" dans la fonction suivante

def sort_ish(L):
    mov,key = L[0]
    sorted_final=[mov]
    k=1
    n=len(L)
    while k<n:
        mov,keyy=L[k]
        if keyy<key:
            return sorted_final
        else:
            sorted_final.append(mov)
            k+=1
    return sorted_final

In [6]:
# name : nom de l'auteur
#database : base de données contenant tous les auteurs
#train : base de données content tous les mouvements
# n : si tf-idf, nombre de mots utilisés dans la liste tf idf
# option 1 : tf-idf pour prédire OU texte entier du mouvement 'tf-idf' ou 'texte'
# option2 : 'sum' ou 'whole' pour savoir si l'on prend pour l'auteur les données du texte de son summary ou du texte de la page entière


def pred(name,database,train,n,option1,option2):
    if option1=='tf-idf':
        #avec l'option tf-idf, on se base sur les mots les plus importants pour chaque mouvement au sens 
        #de la métrique tf-idf
        if option2=='sum':
            txt = database[database['Nom']==name]['sum_lem'].values[0].split()
        elif option2=='whole':
            txt = database[database['Nom']==name]['page_lem'].values[0].split()
        #on prend le texte associé à l'auteur *name*
        
        pred={}     #dictionnaire des prédictions
        m=len(train)
        
        for k in range(m):      #pour chaque mouvement
            mov = train['mouvement'][k] #on récupère son nom
            L= eval(train['tf_idf'][k])[:n] #et les n premiers mots tf_idf (n paramètre de la fction)
            c=0 #compteur qui va compter combien de mots du texte associé à l'auteur (le summary wikipedia) sont des mots présents dans les n tf idf
            for keyword in L:
                #pour chaque mot clé tf idf
                if keyword in txt:
                    #s'il est dans le texte associé à l'auteur, on incrémente c.
                    c+=1
            pred[mov]=c 
            #le nombre de mots est associé au mouvement dans le dictionnaire

        PRED = sorted(pred.items(), key=lambda mot_proba: mot_proba[1])      
        PRED.reverse()
        #on ordonne le dictionnaire selon les clés
        return sort_ish(PRED) #on retourne tous les mots ayant la valeur maximale ! 
    
    
    if option1=='texte':
        #idem que pour tf-idf sauf qu'on s'intéresse à tout le texte lemmatisé de la page du mouvement 
        if option2=='sum':
            txt = database[database['Nom']==name]['sum_lem'].values[0].split()
        elif option2=='whole':
            txt = database[database['Nom']==name]['page_lem'].values[0].split()
        pred={}
        m=len(train)
        for k in range(m):
            mov = train['mouvement'][k]
            L= train['texte_lem'][k].split() #voilà la seule différence : 
            #on prend comme liste de mots L le texte_lem qui est le texte de la page wiki du mouvement.
            c=0
            for keyword in L:
                if keyword in txt:
                    c+=1
            pred[mov]=c
        PRED = sorted(pred.items(), key=lambda mot_proba: mot_proba[1])
        PRED.reverse()
        return sort_ish(PRED)


Je me rends compte que certaines données sont incomplètes : je travaille ainsi pour régler le problème.

In [7]:
data = pd.read_csv('auteurs_completed_lem.csv',index_col=[0])
#Je charge la base de données des auteurs complétée qui comporte désormais 622 entrées.
#Je remarque que certaines ont NaN dans la colomne censée contenir la page wikipédia de l'auteur ainsi que la page wikipédia lemmatisée.
#C'était en fait du au format de certains noms (écrit avec un _ plutôt qu'un - par ex et impossible pour la bibliothèque de trouver la page wikipédia en conséquence)

In [8]:
import wikipedia 
from wikipedia import WikipediaPage
wikipedia.set_lang("fr")
#importation des biblis wiki pour recupérer facilement les données

In [10]:
# premier traitement disparu pour vérifier qu'une entrée est vide : 
# c'était data['is_empty]=data['page'].apply(lambda s : int(type(s)==str))
# data_acompleter = data[data['is_empty']==0]

#data_acompleter -> les auteurs dont on a pas récupéré les pages wikipédia

In [11]:
#data_complete = data[data['is_empty']==1]

In [11]:
def wiki(s):
    return WikipediaPage(s).content
    # associe à une str le contenu de sa page wikipédia associée. On suppose que toutes les entrées correspondent bien à des pages wikipédia (logique ici car les auteurs ont été scrapés sur wikipedia donc tant que rien n'est supprimé, cela fonctionne).

In [12]:
# data_acompleter['page']=data_acompleter['Nom'].apply(wiki)
#je recupère ici 

Je sauvegarde les progrès et charge les données utiles à la prédiction.

In [13]:
#data_acompleter.to_csv('auteurs_incomplete.csv')
data_acompleter=pd.read_csv('auteurs_incomplete_completed.csv',index_col=[0])
#auteurs = pd.concat([data_complete,data_acompleter])
#auteurs.to_csv('auteurs_complete2.csv')
#je sauvegarde puis recharge puis sauvegarde la base de données complète.
auteurs = pd.read_csv('auteurs_complete2.csv')
mouvements=pd.read_csv('Mouvements_wiki_cleaned1.csv',index_col=[0])


#je charge les mouvementsdel auteurs['is_empty']
auteurs['len']=auteurs['page_lem'].apply(len)

authors = list(auteurs['Nom'])

from tqdm import tqdm


In [14]:
database=auteurs
train=mouvements
n=50
#je fixe les arguments pour apporter de la lisibilité.

#database : le dataframe contenant les auteurs, le mouvement associé, le summary de leur page wikipédia, leur page wikipédia, et ces contenus lemmatisés également

In [15]:
m=len(authors)
predictions = [[] for k in range(m)]

L'exécution de la prochaine cellule prend environ 30 minutes. 
Les dataframes auteurs et AUT sont cependant stockés dans les .csv auteurs_pred1.csv et auteurs_pred2.csv. 

In [74]:
# Je choisis chaque combinaison d'option une fois
OPT1 = ['tf-idf','texte'] # j'utilise un comptage de mots similaires simples ou bien le comptage de mots en commun avec ceux ressortis par tf-idf
OPT2 = ['whole','sum']  # je compare au summary lemmatisé de la page wikipédia ou bien à la page wikipédia lemmatisée entière

for k in tqdm(range(m)): #pour tous les auteurs
    L=[]
    for i in range(2):
        for j in range(2):
            name = authors[k]
            option1 = OPT1[i]
            option2 = OPT2[j]
            a=pred(name,database,train,n,option1,option2)
            L.append(a)
    predictions[k]=L
            #la liste contient des listes comportant les prédictions avec les différentes options pour chaque auteur        

100%|████████████████████████████████████████████████████████████████████████████████| 622/622 [12:37<00:00,  1.22s/it]


In [1]:
auteurs['pred']=predictions #je rajoute la prédiction au dataframe.
# l'ajouter directement avec la méthode .apply m'aurait semblé un peu fouilli, il aurait fallu fixer les arguments dans la fonction ou écrire une autre fonction auxiliaire.

In [76]:
auteurs

,Nom,Mouvement,summary,sum_lem,mvm_label,page,page_lem,len,pred
0,William S. Burroughs,Beat Generation,William Seward Burroughs (/ˈwɪljəm ˈsuɚd ˈbɜɹo...,William Seward Burroughs ˈsuɚd dit William Bur...,0,"William Seward Burroughs II (; February 5, 191...",William Seward Burroughs ii February 5 1914 au...,61923,"[[Beat Generation], [Beat Generation], [symbol..."
1,Lucien Carr,Beat Generation,"Lucien Carr, né le 1er mars 1925 à New York et...",Lucien Carr naître premier mars 1925 New York ...,0,"Lucien Carr (March 1, 1925 – January 28, 2005)...",Lucien Carr March 1 1925 January 28 2005 was a...,18339,"[[Beat Generation], [Beat Generation, Générati..."
3,John Clellon Holmes,Beat Generation,"John Clellon Holmes est un romancier, essayist...",John Clellon holme romancier essayiste poète a...,0,"John Clellon Holmes (March 12, 1926, Holyoke, ...",John Clellon Holmes March 12 1926 Holyoke Mass...,2592,"[[Beat Generation], [Génération perdue], [Beat..."
4,Gregory Corso,Beat Generation,"Gregory Nunzio Corso (New York, 26 mars 1930 -...",gregory Nunzio Corso New York 26 mars 1930 Min...,0,"Gregory Nunzio Corso (March 26, 1930 – January...",Gregory Nunzio Corso March 26 1930 January 17 ...,40570,"[[Beat Generation], [Beat Generation], [symbol..."
5,Lenore Kandel,Beat Generation,"Lenore Kandel, née le 14 janvier 1932 à New Yo...",Lenore Kandel naître 14 janvier 1932 New York ...,0,"Lenore Kandel (January 14, 1932 in New York Ci...",Lenore Kandel January 14 1932 in New York City...,5128,"[[Beat Generation], [Beat Generation], [Beat G..."
...,...,...,...,...,...,...,...,...,...
573,Louis_Guillaume,Ecole de Rochefort,Louis Guillaume (1907 - 1971) est un écrivain ...,Louis Guillaume 1907 1971 écrivain poète franç...,2,Louis Guillaume (1907 - 1971) est un écrivain ...,Louis Guillaume 1907 1971 écrivain poète franç...,5062,"[[Parnasse], [pleiade], [symboliste], [symboli..."
576,Gabriel_Audisio_(romancier_et_poète),Ecole de Rochefort,Gabriel Audisio est un écrivain et poète franç...,Gabriel Audisio écrivain poète français naître...,2,Gabriel Audisio est un écrivain et poète franç...,Gabriel Audisio écrivain poète français naître...,5139,"[[Ecole de Rochefort], [Imagisme, pleiade], [s..."
595,Jean-Vincent_Verdonnet,Ecole de Rochefort,"Jean-Vincent Verdonnet, né le 19 avril 1923 à ...",Verdonnet naître 19 avril 1923 Bossey décéder ...,2,"Jean-Vincent Verdonnet, né le 19 avril 1923 à ...",Verdonnet naître 19 avril 1923 Bossey décéder ...,2041,"[[Ecole de Rochefort], [pleiade], [symboliste]..."
598,Alain_Robbe-Grillet,Nouveau Roman,"Alain Robbe-Grillet, né le 18 août 1922 à Bres...",Alain Robbe Grillet naître 18 août 1922 brest ...,8,"Alain Robbe-Grillet, né le 18 août 1922 à Bres...",Alain Robbe Grillet naître 18 août 1922 brest ...,7079,"[[Nouveau Roman], [Nouveau Roman], [symboliste..."


In [77]:
auteurs['pred_tf_w']=auteurs['pred'].apply(lambda L: L[0])
auteurs['pred_tf_sum']=auteurs['pred'].apply(lambda L: L[1])
auteurs['pred_txt_w']=auteurs['pred'].apply(lambda L: L[2])
auteurs['pred_txt_s']=auteurs['pred'].apply(lambda L: L[3])

# je mets chaque prédictions dans une seule colonne.


In [79]:
auteurs[['Mouvement','pred_tf_w','pred_tf_sum','pred_txt_w','pred_txt_s']]

#résultat plus lisibile

,Mouvement,pred_tf_w,pred_tf_sum,pred_txt_w,pred_txt_s
0,Beat Generation,[Beat Generation],[Beat Generation],[symboliste],[symboliste]
1,Beat Generation,[Beat Generation],"[Beat Generation, Génération perdue]",[symboliste],[symboliste]
3,Beat Generation,[Beat Generation],[Génération perdue],[Beat Generation],[symboliste]
4,Beat Generation,[Beat Generation],[Beat Generation],[symboliste],[Beat Generation]
5,Beat Generation,[Beat Generation],[Beat Generation],[Beat Generation],[Beat Generation]
...,...,...,...,...,...
573,Ecole de Rochefort,[Parnasse],[pleiade],[symboliste],[symboliste]
576,Ecole de Rochefort,[Ecole de Rochefort],"[Imagisme, pleiade]",[symboliste],[symboliste]
595,Ecole de Rochefort,[Ecole de Rochefort],[pleiade],[symboliste],[symboliste]
598,Nouveau Roman,[Nouveau Roman],[Nouveau Roman],[symboliste],[symboliste]


Nouveau problème (léger) dans les données. Il se trouve que Pierre a réalisé le scraping de la liste des auteurs et moi celle des mouvements et aucun de nous deux n'avait vu que certains auteurs scrapés par Pierre ne correspondaient à aucun des mouvements du tableau wiki_mouvements. Par conséquent, impossible de réaliser des prédictions correctes. 
Je vais donc les identifier et les retirer.

In [81]:
mouvements_auteurs = list(set(auteurs.Mouvement.values))

In [85]:
L=[]
for mov in mouvements_auteurs:
    if mov not in list(set(mouvements.mouvement)):
        L.append(mov)

        # si un mouvement est dans la liste des mouvements associés aux auteurs mais pas dans le tableau des mouvements, je le récupère dans la liste L.

In [87]:
auteurs['del']=auteurs.Mouvement.apply(lambda mov: int(mov in L)) 
AUT = auteurs[auteurs['del']==0] #les auteurs qui ne sont pas dans la liste décrite précédemment
AUT=AUT[['Mouvement','pred_tf_w','pred_tf_sum','pred_txt_w','pred_txt_s']] 
#Le nouveau tableau a 559 entrée.

In [126]:
#AUT['test']=AUT['pred_tf_w'].apply(len)
#AUT['Mouvement'][0]  == AUT['pred_tf_w'][0][0]

#tests pour vérifier la cohérence

<ipython-input-126-72f5baad327e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AUT['test']=AUT['pred_tf_w'].apply(len)


In [16]:
AUT = pd.read_csv('auteurs_pred2.csv') # Je le recharge directement sinon toutes les opérations suivantes sont impossibles.

In [17]:
AUT.index = [k for k in range(len(AUT))] #je réinitialise l'index.

In [18]:
accuracy_tf_w = np.array([ int(AUT['Mouvement'][k] in AUT['pred_tf_w'][k]) for k in range(559)])

# je crée un np.array qui vérifie si la valeur 'Mouvement' de chaque auteur appartient au(x) mouvement(s) prédit(s) (rappel que certains mouvements)
#peuvent être ex aequos, auquel cas on les considère tous valables). Pour chaque auteur en position k dans le tableau, 
# accuracy_tf_w[k]  vaudra 1 si c'est le cas, 0 sinon.


In [19]:
accuracy_tf_w.sum()/559 

#On réalise donc un calcul de l'accuracy : % de bonnes prédictions


0.3881932021466905

In [20]:
accuracy_tf_sum = np.array([ int(AUT['Mouvement'][k] in AUT['pred_tf_sum'][k]) for k in range(559)])

In [21]:
accuracy_tf_sum.sum()/559

0.4597495527728086

In [22]:
accuracy_txt_w = np.array([ int(AUT['Mouvement'][k] in AUT['pred_txt_w'][k]) for k in range(559)])

In [23]:
accuracy_txt_w.sum()/559

0.06976744186046512

In [24]:
accuracy_txt_s = np.array([ int(AUT['Mouvement'][k] in AUT['pred_txt_s'][k]) for k in range(559)])

In [25]:
accuracy_txt_s.sum()/559

0.10912343470483005

In [ ]:
# on réalise cela pour toutes les méthodes. et on sauvegarde.

In [144]:
auteurs.to_csv('auteurs_pred_1.csv')

In [145]:
AUT.to_csv('auteurs_pred2.csv')

On obtient des résultats bien différents, une accuracy allant de 0.07 à 0.45.

Quelques commentaires : 
- pour rappel, il y a une petite liberté prise avec la définition de l'accuracy car des prédictions peuvent parfois être ex aequo et on considère la prédiction bonne si une des prédictions ex aequo est la bonne. Quantité de prédictions "multiples" : 

- à type de comptage égal (tf-idf ou simple comptage de mots communs entre les pages lemmatisées), on a une meilleure accuracy en considérant seulement les summaries des pages des auteurs ( 0.11 > 0.07 et 0.45>0.39)

- à texte comparé égal (summary ou texte entier, les deux lemmatisés), l'usage du comptage de mots communs entre la page et les mots clés tf-idf augmente *significativement* l'accuracy. De 0.07 à 0.39 et de 0.11 à 0.45. 


On en déduit que l'usage de l'évaluation tf-idf pour retirer les mots les plus significatifs sur chaque mouvement est très efficace par rapport au simple comptage. En effet, même en retirant les stop words, des mots comme auteurs, romans etc pourront aisément être communs aux différentes pages sans que cela soit significatif en ce que cela ne caractérise pas les mouvements. Ainsi, une page plus longue pourrait facilement avoir plus de mots communs avec celle d'un auteur, simplement car plus la page est longue, plus ou pourront retrouver ces mots. On peut aussi penser que c'est pour cela qu'utiliser les summaries augmente le score. 